# Part1: Langchain 기초


## 1. 환경 구성

### 1) 라이브러리 설치

In [1]:
!pip install -q langchain langchain-openai tiktoken

### 2) OpenAI 인증키 설정
https://openai.com/

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

## 2. LLM Chain

### 1) Prompt + LLM


In [24]:
from langchain_openai import ChatOpenAI

# model
llm = ChatOpenAI(model="gpt-4o-mini")

# chain 실행
result = llm.invoke("지구의 자전 주기는?")

In [ ]:
result.content

'지구의 자전 주기는 약 24시간입니다. 정확히 말하자면, 지구가 한 번 자전하는 데 걸리는 시간은 약 23시간 56분 4초로, 이를 "항성일"이라고 합니다. 그러나 지구가 태양 주위를 공전하면서 자전하기 때문에, 태양을 기준으로 한 하루인 "태양일"은 약 24시간으로 정의됩니다.'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
prompt

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

# chain 연결 (LCEL)
chain = prompt | llm

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
# llm = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

# LCEL chaining
chain = prompt | llm | output_parser

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

"지구의 자전 주기는 약 24시간입니다. 더 정확히 말하면, 지구가 한 번 자전하는 데 걸리는 시간은 약 23시간 56분 4초로, 이를 '항성일'이라고 합니다. 그러나 우리가 일반적으로 사용하는 24시간은 태양이 하늘에서 같은 위치에 돌아오는 데 걸리는 시간인 '태양일'을 기준으로 합니다. 이 차이는 지구가 공전하면서 태양의 위치가 조금씩 이동하기 때문에 발생합니다."

### 2) Multiple Chains

In [ ]:
prompt1 = ChatPromptTemplate.from_template("translates {korean_word} to English.")
prompt2 = ChatPromptTemplate.from_template(
    "explain {english_word} using oxford dictionary to me in Korean."
)

# llm = ChatOpenAI(model="gpt-4o-mini")

chain1 = prompt1 | llm | StrOutputParser()

chain1.invoke({"korean_word":"미래"})

'The Korean word "미래" translates to "future" in English.'

In [15]:
chain2 = (
    {"english_word": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)

chain2.invoke({"korean_word":"미래"})

'"미래"는 영어로 "future"로 번역됩니다. 옥스퍼드 사전에 따르면, "future"는 "어떤 일이 일어나거나 어떤 상태가 발생할 가능성이 있는 시간"을 의미합니다. 즉, 미래는 우리가 아직 경험하지 않았지만 앞으로 일어날 수 있는 사건이나 상황을 나타냅니다. 이 단어는 시간의 개념을 포함하며, 개인의 계획이나 세계의 변화와 관련하여 자주 사용됩니다.'

## 3. Prompt
  

### 1) PromptTemplate

In [16]:
from langchain_core.prompts import PromptTemplate

# 'name'과 'age'라는 두 개의 변수를 사용하는 프롬프트 템플릿을 정의
template_text = "안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(name="홍길동", age=30)

filled_prompt

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.'

In [17]:
# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n아버지를 아버지라 부를 수 없습니다.")
              + "\n\n{language}로 번역해주세요."
)

combined_prompt

PromptTemplate(input_variables=['age', 'language', 'name'], template='안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n{language}로 번역해주세요.')

In [18]:
combined_prompt.format(name="홍길동", age=30, language="영어")

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n영어로 번역해주세요.'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# llm = ChatOpenAI(model="gpt-4o-mini")
chain = combined_prompt | llm | StrOutputParser()
chain.invoke({"age":30, "language":"영어", "name":"홍길동"})

'Hello, my name is Hong Gildong and I am 30 years old.\n\nI cannot call my father "father."'

### 2) ChatPromptTemplate

In [19]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
messages

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.'),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?')]

In [20]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

'태양계에서 가장 큰 행성은 목성(Jupiter)입니다. 목성은 지구의 약 1,300배에 달하는 크기를 가지고 있으며, 주로 수소와 헬륨으로 구성되어 있습니다. 또한, 목성은 강력한 자기장과 수많은 위성을 가지고 있는 특징이 있습니다.'

### 3) Message

In [21]:
# MessagePromptTemplate 활용

from langchain_core.prompts import SystemMessagePromptTemplate,  HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("이 시스템은 천문학 질문에 답변할 수 있습니다."),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)


messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
messages

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.'),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?')]

In [ ]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

'태양계에서 가장 큰 행성은 목성입니다. 목성은 태양 주위를 도는 행성 중에서 가장 크고 질량도 가장 많이 가지고 있습니다.'

## 5. Model Parameter


### 1) 모델 클래스 유형

#### LLM

In [25]:
from langchain_openai import OpenAI

llm = OpenAI(model='gpt-4o-mini')

llm.invoke("한국의 대표적인 관광지 3군데를 추천해주세요.")

' (Please recommend three representative tourist attractions in Korea.)"\n\nKorea is full of wonderful tourist attractions. Here are three representative ones:\n\n1. **Gyeongbokgung Palace (경복궁)** - Located in Seoul, Gyeongbokgung is the largest and most iconic of the Five Grand Palaces built during the Joseon Dynasty. Visitors can explore the beautiful architecture, gardens, and the National Palace Museum of Korea located on the palace grounds. Don\'t miss the changing of the guard ceremony at the main gate!\n\n2. **Jeju Island (제주도)** - Known for its stunning natural beauty, Jeju Island is a UNESCO World Heritage site with unique volcanic landscapes, beautiful beaches, and the famous Hallasan Mountain. It\'s a great place for outdoor activities such as hiking, exploring lava tubes, and enjoying local cuisine. \n\n3. **Bukchon Hanok Village (북촌한옥마을)** - This traditional village located between Gyeongbokgung Palace and Changdeokgung Palace features hundreds of well-preserved hanoks (tr

#### ChatModel

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model='gpt-4o-mini')

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 여행 전문가입니다."),
    ("user", "{user_input}"),
])

chain = chat_prompt | chat
chain.invoke({"user_input": "안녕하세요? 한국의 대표적인 관광지 3군데를 추천해주세요."})

AIMessage(content='안녕하세요! 한국의 대표적인 관광지 3곳을 추천해드릴게요.\n\n1. **경복궁 (Gyeongbokgung Palace)**:\n   서울에 위치한 경복궁은 조선 왕조의 주요 궁궐로, 아름다운 전통 건축과 정원이 인상적입니다. 궁궐 내부에는 국립민속박물관과 국립고궁박물관도 있어 한국의 역사와 문화를 깊이 있게 이해할 수 있습니다. 특히, 매일 열리는 수문장 교대식은 많은 관광객들에게 인기가 있습니다.\n\n2. **제주도 (Jeju Island)**:\n   제주도는 자연 경관이 아름답고 다양한 액티비티를 즐길 수 있는 인기 있는 여행지입니다. 한라산, 성산 일출봉, 만장굴 등 자연 명소가 많으며, 제주 고유의 문화와 음식도 매력적입니다. 또한, 해변에서의 여유로운 시간이나 다양한 수상 스포츠를 즐길 수 있습니다.\n\n3. **부산 (Busan)**:\n   부산은 한국의 두 번째 도시로, 아름다운 해변과 풍부한 해산물로 유명합니다. 해운대와 광안리 해수욕장은 여름철 많은 관광객이 찾는 인기 명소이며, 자갈치 시장에서 신선한 해산물을 맛볼 수 있습니다. 또한, 부산 타워와 감천문화마을 등 다양한 관광명소가 있어 즐길 거리가 많습니다.\n\n이 세 곳은 각각의 매력이 있어 한국의 다양한 문화를 경험할 수 있는 좋은 장소입니다. 즐거운 여행 되세요!', response_metadata={'token_usage': {'completion_tokens': 352, 'prompt_tokens': 35, 'total_tokens': 387, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt

### 2) 모델 파라미터 설정


#### 모델에 직접 파라미터를 전달 (모델 생성 시점)

In [27]:
from langchain_openai import ChatOpenAI

# 모델 파라미터 설정
params = {
    "temperature": 0.7,         # 생성된 텍스트의 다양성 조정
    "max_tokens": 100,          # 생성할 최대 토큰 수
}

kwargs = {
    "frequency_penalty": 0.5,   # 이미 등장한 단어의 재등장 확률
    "presence_penalty": 0.5,    # 새로운 단어의 도입을 장려
    "stop": ["\n"]              # 정지 시퀀스 설정

}

# 모델 인스턴스를 생성할 때 설정
model = ChatOpenAI(model="gpt-4.1-mini", **params, model_kwargs = kwargs)


# 모델 호출
question = "태양계에서 가장 큰 행성은 무엇인가요?"
response = model.invoke(input=question)

# 전체 응답 출력
print(response)

content='태양계에서 가장 큰 행성은 목성입니다. 목성은 지름이 약 139,820km로, 태양계 내 다른 행성들보다 훨씬 큽니다.' response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 20, 'total_tokens': 63, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini', 'system_fingerprint': 'fp_3f58d112f7', 'finish_reason': 'stop', 'logprobs': None} id='run-249eb887-c5b9-4db6-81df-c593600f622a-0'


#### 모델에 직접 파라미터를 전달 (모델 호출 시점)

In [28]:
# 모델 파라미터 설정
params = {
    "temperature": 0.7,         # 생성된 텍스트의 다양성 조정
    "max_tokens": 10,          # 생성할 최대 토큰 수
}

# 모델 인스턴스를 호출할 때 전달
response = model.invoke(input=question, **params)

# 문자열 출력
print(response.content)

태양계에서 가장 큰 행성은 목


#### 모델에 추가적인 파라미터를 전달
- bind 메서드를 사용

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

model = ChatOpenAI(model="gpt-4o-mini", max_tokens=100)

messages = prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")

before_answer = model.invoke(messages)

# # binding 이전 출력
print(before_answer)

# 모델 호출 시 추가적인 인수를 전달하기 위해 bind 메서드 사용 (응답의 최대 길이를 10 토큰으로 제한)
chain = prompt | model.bind(max_tokens=10)

after_answer = chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

# binding 이후 출력
print(after_answer)

content='가장 큰 행성은 목성입니다. 목성은 태양계에서 가장 크고 질량이 가장 큰 행성으로, 지름은 약 14만 2000km에 달합니다.'
content='태양계에서 가장 큰'


## 6. Output Parsers


### 1) CSV Parser

In [33]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [34]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | output_parser

chain.invoke({"subject": "popular Korean cusine"})

['Kimchi', 'Bibimbap', 'Bulgogi', 'Tteokbokki', 'Japchae']

### 2) JSON Parser

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# 자료구조 정의 (pydantic)
class CusineRecipe(BaseModel):
    name: str = Field(description="name of a cusine")
    recipe: str = Field(description="recipe to cook the cusine")

In [ ]:
# 출력 파서 정의
output_parser = JsonOutputParser(pydantic_object=CusineRecipe)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}
```


In [ ]:
# prompt 구성
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

print(prompt)

input_variables=['query'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}\n```'} template='Answer the user query.\n{format_instructions}\n{query}\n'


In [ ]:
chain = prompt | model | output_parser

chain.invoke({"query": "Let me know how to cook Bibimbap"})

{'name': 'Bibimbap',
 'recipe': 'Bibimbap is a Korean mixed rice dish. To cook Bibimbap, you will need the following ingredients: cooked rice, vegetables (such as spinach, bean sprouts, carrots, zucchini, mushrooms), beef or tofu, eggs, sesame oil, soy sauce, sugar, garlic, and Korean red pepper paste (gochujang). To prepare Bibimbap, sauté the vegetables separately,'}